<a href="https://colab.research.google.com/github/saotomryo/Use_MMSegmentation/blob/main/Use_MMSegmentation_deeplabv3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MMSegmentationを利用したSegmentationタスク学習の実施
## MMCVのインストール

In [ ]:
!pip install -U openmim
!mim install mmcv-full

## MMSegmentationのインストール

In [ ]:
!rm -rf mmsegmentation
!git clone https://github.com/open-mmlab/mmsegmentation.git 
%cd mmsegmentation
!pip install -e .

In [5]:
# Check Pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# Check MMSegmentation installation
import mmseg
print(mmseg.__version__)

1.12.0+cu113 True
0.27.0


In [22]:
import os.path as osp
import numpy as np
from PIL import Image
import mmcv
import matplotlib.pyplot as plt
device = "cuda" if torch.cuda.is_available() else "cpu"
print("device = ", device)
# convert dataset annotation to semantic segmentation map
data_root = # データのルートフォルダを指定します。
img_dir = # 学習用データの格納フォルダ
ann_dir = # アノテーションファイルの格納フォルダ

device =  cuda


## オリジナルデータセットの設定

データ設定の作成や設定については、以下の記事を参照ください。
https://qiita.com/ryosao/items/d753a7d2c01f4c3f520f

In [16]:
## 分類したいクラスの情報
classes = ('background','builing')

## クラスに対応するアノテーションの色を指定
palette = [[0,0,0],[255, 0, 0]]


## 利用するオリジナルのデータセットの定義

In [ ]:
from mmseg.datasets.builder import DATASETS
from mmseg.datasets.custom import CustomDataset

@DATASETS.register_module()
class OriginalDataset(CustomDataset):
    CLASSES = classes
    PALETTE = palette
    def __init__(self, split, **kwargs):
        super().__init__(img_suffix='.png', seg_map_suffix='.png', 
                     split=split, **kwargs)
        assert osp.exists(self.img_dir) and self.split is not None

## コンフィグファイルの設定

In [31]:
from mmcv import Config
cfg = Config.fromfile('configs/deeplabv3plus/deeplabv3plus_r50-d8_512x512_20k_voc12aug.py')

In [ ]:
from mmseg.apis import set_random_seed

# Since we use ony one GPU, BN is used instead of SyncBN
cfg.norm_cfg = dict(type='BN', requires_grad=True)
cfg.model.backbone.norm_cfg = cfg.norm_cfg
cfg.model.decode_head.norm_cfg = cfg.norm_cfg
cfg.model.auxiliary_head.norm_cfg = cfg.norm_cfg

# 分類するクラス数
cfg.model.decode_head.num_classes = 2  # 分類したいクラス数
cfg.model.auxiliary_head.num_classes = 2  # 分類したいクラス数

# Modify dataset type and path
cfg.dataset_type = 'OriginalDataset' # 作成したデータセットクラスのクラス名
cfg.data_root = data_root # データのルートフォルダ

# バッチサイズの設定
cfg.data.samples_per_gpu = 4
cfg.data.workers_per_gpu=4


cfg.data.train.type = cfg.dataset_type
cfg.data.train.data_root = cfg.data_root
cfg.data.train.img_dir = img_dir # 学習データの格納フォルダ
cfg.data.train.ann_dir = ann_dir # 学習データのアノテーションファイルの格納フォルダ
cfg.data.train.pipeline = cfg.train_pipeline
cfg.data.train.split = 'splits/train.txt'

cfg.data.val.type = cfg.dataset_type
cfg.data.val.data_root = cfg.data_root
cfg.data.val.img_dir = img_dir # 検証データの格納フォルダ
cfg.data.val.ann_dir = ann_dir  # 検証データのアノテーションファイルの格納フォルダ
cfg.data.val.pipeline = cfg.test_pipeline
cfg.data.val.split = 'splits/val.txt'

cfg.data.test.type = cfg.dataset_type
cfg.data.test.data_root = cfg.data_root
cfg.data.test.img_dir = img_dir # テストデータの格納フォルダ
cfg.data.test.ann_dir = ann_dir  # テストデータのアノテーションファイルの格納フォルダ
cfg.data.test.pipeline = cfg.test_pipeline
cfg.data.test.split = 'splits/val.txt'

# Set up working dir to save files and logs.
cfg.work_dir = './work_dirs/tutorial'

# 学習のエポック数等の設定
#cfg.runner.max_iters = 200
#cfg.log_config.interval = 10
#cfg.evaluation.interval = 200
#cfg.checkpoint_config.interval = 200

# Set seed to facitate reproducing the result
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)

cfg.device = device

# Let's have a look at the final config used for training
print(f'Config:\n{cfg.pretty_text}')

In [ ]:
from mmseg.datasets import build_dataset
from mmseg.models import build_segmentor
from mmseg.apis import train_segmentor


# Build the dataset
datasets = [build_dataset(cfg.data.train)]

# Build the detector
model = build_segmentor(
    cfg.model, train_cfg=cfg.get('train_cfg'), test_cfg=cfg.get('test_cfg'))
# Add an attribute for visualization convenience
model.CLASSES = datasets[0].CLASSES

# Create work_dir
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
train_segmentor(model, datasets, cfg, distributed=False, validate=True, 
                meta=dict())

In [ ]:
from mmseg.apis import inference_segmentor, init_segmentor, show_result_pyplot
from mmseg.core.evaluation import get_palette

img = mmcv.imread('確認する画像のパス')

model.cfg = cfg
result = inference_segmentor(model, img)
plt.figure(figsize=(8, 6))
show_result_pyplot(model, img, result, palette)